In [ ]:
import pandas as pd
import mplfinance as mpf
import matplotlib.dates as mdates
import numpy as np

# Load the MT5 CSV
df = pd.read_csv( "xauusdm15.filtered.csv" )

# Set DateTime as index
df['DateTime'] = pd.to_datetime(df["DateTime"])
df.set_index("DateTime", inplace=True)

# Optional: keep only OHLC + Volume for plotting
ohlc_df = df[["Open", "High", "Low", "Close", "Volume"]]


In [ ]:
import pandas as pd

def detect_macd_cross(df, macd_col='MACD', signal_col='MACD_Signal'):
    """
    Detects MACD crossovers in a DataFrame.

    Parameters:
    - df: DataFrame containing MACD and Signal columns
    - macd_col: name of the MACD column
    - signal_col: name of the Signal column

    Returns:
    - A Series with 1 (buy), -1 (sell), 0 (no crossover)
    """
    cross = pd.Series(0, index=df.index)

    # Buy signal: MACD crosses above Signal
    buy_mask = (df[macd_col].shift(1) < df[signal_col].shift(1)) & (df[macd_col] > df[signal_col])
    cross[buy_mask] = 1

    # Sell signal: MACD crosses below Signal
    sell_mask = (df[macd_col].shift(1) > df[signal_col].shift(1)) & (df[macd_col] < df[signal_col])
    cross[sell_mask] = -1

    return cross


In [ ]:
import talib
import pandas as pd

# MACD calculation
macd_line, signal_line, macd_hist = talib.MACD(
    ohlc_df['Close'],
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)

ohlc_df['MACD'] = macd_line
ohlc_df['MACD_Signal'] = signal_line
ohlc_df['MACD_Hist'] = macd_hist

# Detect MACD crossover
ohlc_df['Crossover'] = detect_macd_cross(ohlc_df)

# Add SL and TP columns
ohlc_df['SL'] = pd.NA
ohlc_df['TP'] = pd.NA

# Bullish crossover: SL = previous candle low, TP = Close + 20
ohlc_df.loc[ohlc_df['Crossover'] == 1, 'SL'] = ohlc_df['Low'].shift(1)
ohlc_df.loc[ohlc_df['Crossover'] == 1, 'TP'] = ohlc_df['Close'] + 20

# Optional: bearish crossover
# ohlc_df.loc[ohlc_df['Crossover'] == -1, 'SL'] = ohlc_df['High'].shift(1)
# ohlc_df.loc[ohlc_df['Crossover'] == -1, 'TP'] = ohlc_df['Close'] - 20

# Filter bullish crossovers
cross_rows = ohlc_df[ohlc_df['Crossover'] == 1]

# Display
cross_rows[['MACD', 'MACD_Signal', 'MACD_Hist', 'Crossover', 'SL', 'TP']].tail()


In [ ]:
import mplfinance as mpf

# Prepare additional plots for SL and TP
addplots = [
    mpf.make_addplot(ohlc_df['SL'], type='scatter', markersize=50, marker='v', color='red'),
    mpf.make_addplot(ohlc_df['TP'], type='scatter', markersize=50, marker='^', color='green')
]

# Plot candlestick chart with SL and TP
mpf.plot(
    ohlc_df.iloc[23300:],
    type='candle',
    style='charles',
    addplot=addplots,
    title='Candles with SL/TP',
    ylabel='Price',
    volume=False
)


In [ ]:
import mplfinance as mpf
import pandas as pd

def plot_candles(df):
    df = df.copy()
    df.index = pd.to_datetime(df.index)

    addplots = []

    # Only add EMA if it exists
    # addplots.append(mpf.make_addplot(df['EMA_51'], color='orange', width=1.2))

    addplots.append(mpf.make_addplot(df['MACD'], panel=1, color='green', width=1.2))
    addplots.append(mpf.make_addplot(df['MACD_Signal'], panel=1, color='red', width=1.2))

    charttype = "candle"
    title = f"{charttype.capitalize()} - Chart"

    fig, axes = mpf.plot(
        df,
        type=charttype,
        style='classic',
        volume=False,
        figratio=(14, 7),
        figscale=1.2,
        returnfig=True,
        addplot=addplots,
        title=title,
        block=False
    )
    return fig, axes


In [ ]:
def vertical_lines_on(axes, df, panel_idx=1, flag_col='isBear', color='red', linewidth=2):

    # Handle both list of axes or single axes
    ax = axes[panel_idx] if isinstance(axes, (list, tuple)) else axes

    ylim = ax.get_ylim()
    for i, flag in enumerate(df[flag_col]):
        if flag == 1:
            ax.vlines(i, ylim[0], ylim[1], color=color, linewidth=linewidth)
    ax.figure.canvas.draw()


In [ ]:
dta = ohlc_df.iloc[23300:];

fig, axis = plot_candles(dta);
vertical_lines_on(axis, dta, 0, "Crossover")